In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from keras.datasets import imdb
from tensorflow.keras.layers import Dense ,Input, Flatten, Embedding, LSTM

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
num_words = 15000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = 15000)

17464789/17464789 [==============================] - 0s 0us/step


In [3]:
x_train = tf.keras.utils.pad_sequences(x_train,maxlen = 2494)
x_test = tf.keras.utils.pad_sequences(x_test,maxlen = 2494)
max_seq_len = x_train.shape[1]
x_test.shape

(25000, 2494)

In [4]:
def get_review(x_index, train = 0):
    word_to_index = imdb.get_word_index()
    index_dict = dict([(value,key) for (key,value) in word_to_index.items()]) 
    
    if train == 0 :
        review = " ".join([index_dict.get(i -3, "") for i in x_train[x_index]])
        print(review )
        print(y_train[x_index], end = "\t")
    else :
        review = " ".join([index_dict.get(i -3, "") for i in x_test[x_index]])
        print(review)
        print(y_test[x_index], end = "\t")
    
    

In [5]:
get_review(60)

1641221/1641221 [==============================] - 0s 0us/step
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [6]:
inputs = Input(shape = (max_seq_len,))
layer1 = Embedding(15000,128)(inputs)
layer2 = LSTM(32)(layer1)
layer3 = Dense(16, activation='relu')(layer2)
layer4 = Dense(1, activation='sigmoid')(layer3)
lstm_model = Model(inputs=inputs, outputs=layer4)

lstm_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2494)]            0         
                                                                 
 embedding (Embedding)       (None, 2494, 128)         1920000   
                                                                 
 lstm (LSTM)                 (None, 32)                20608     
                                                                 
 dense (Dense)               (None, 16)                528       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,941,153
Trainable params: 1,941,153
Non-trainable params: 0
_________________________________________________________________


In [7]:
lstm_model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=["accuracy"])
lstm_model.fit(x_train,y_train,batch_size=128,epochs = 5)

Epoch 1/5
196/196 [==============================] - 49s 217ms/step - loss: 0.5152 - accuracy: 0.7412
Epoch 2/5
196/196 [==============================] - 39s 199ms/step - loss: 0.3140 - accuracy: 0.8744
Epoch 3/5
196/196 [==============================] - 36s 185ms/step - loss: 0.2487 - accuracy: 0.9034
Epoch 4/5
196/196 [==============================] - 36s 182ms/step - loss: 0.2109 - accuracy: 0.9204
Epoch 5/5
196/196 [==============================] - 34s 173ms/step - loss: 0.1773 - accuracy: 0.9340


In [9]:
train_loss1, train_acc1= lstm_model.evaluate(x_train,  y_train, verbose=2)
test_loss, test_acc= lstm_model.evaluate(x_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc * 100)

782/782 - 25s - loss: 0.1605 - accuracy: 0.9410 - 25s/epoch - 32ms/step
782/782 - 25s - loss: 0.3447 - accuracy: 0.8634 - 25s/epoch - 32ms/step

Test accuracy: 86.33999824523926


In [10]:
 predictions = lstm_model.predict(x_test)


782/782 [==============================] - 25s 31ms/step


In [11]:
for i, item in enumerate(predictions):
    get_review(i,1)
    
    if item >= 0.5:
        print("positive")
        
    else:
        print("negative")
        
    if i == 20:
        break